In [63]:
from dotenv import load_dotenv
import os
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [64]:
# Load from environment
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'

# Global constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [65]:
kg = Neo4jGraph(
    url=NEO4J_URI, 
    username=NEO4J_USERNAME, 
    password=NEO4J_PASSWORD, 
    database=NEO4J_DATABASE
)

### Read the collection of Form 13s

- Investment management firms must report on their investments in companies to the SEC by filing a document called Form 13
- Let's load a collection of Form 13 for managers that have invested in NetApp

In [66]:
import csv

form13s = []

with open('form13.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader: # each row will be a dictionary
        form13s.append(row)

In [67]:
print(f'Length of form 13s: {len(form13s)}')
print(f'Top 5 form 13s: {form13s[0:5]}')

Length of form 13s: 561
Top 5 form 13s: [{'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt', 'managerCik': '1000275', 'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5', 'managerName': 'Royal Bank of Canada', 'reportCalendarOrQuarter': '2023-06-30', 'cusip6': '64110D', 'cusip': '64110D104', 'companyName': 'NETAPP INC', 'value': '64395000000.0', 'shares': '842850'}, {'source': 'https://sec.gov/Archives/edgar/data/1002784/0001387131-23-009542.txt', 'managerCik': '1002784', 'managerAddress': '1875 Lawrence Street, Suite 300, Denver, CO, 80202-1805', 'managerName': 'SHELTON CAPITAL MANAGEMENT', 'reportCalendarOrQuarter': '2023-06-30', 'cusip6': '64110D', 'cusip': '64110D104', 'companyName': 'NETAPP INC', 'value': '2989085000.0', 'shares': '39124'}, {'source': 'https://sec.gov/Archives/edgar/data/1007280/0001007280-23-000008.txt', 'managerCik': '1007280', 'managerAddress': '277 E TOWN ST, COLUMBUS, OH, 43215', 'managerName': 'PUBLIC EMPL

### Create company nodes in the graph

- Use the companies identified in the Form 13s to create Company nodes
- For now, there is only one company - NetApp

In [68]:
# lets work with just the first form for now
first_form13 = form13s[0]

kg.query("""
merge (com: Company {cusip6: $cusip6})
    on create 
        set com.companyName = $companyName,
            com.cusip = $cusip
""", params={
    'cusip6': first_form13['cusip6'],
    'companyName': first_form13['companyName'],
    'cusip': first_form13['cusip']
})

[]

In [69]:
kg.query('match (c:Company) return c limit 1')

[{'c': {'cusip': '64110D104',
   'companyName': ['Netapp Inc', 'NETAPP INC'],
   'cusip6': '64110D'}}]

- Update the company name to match Form 10-K

In [70]:
kg.query("""
    match (com: Company), (form: Form)
    where com.cusip6 = form.cusip6
    return com.companyName, form.names
""")

[{'com.companyName': ['Netapp Inc', 'NETAPP INC'],
  'form.names': ['Netapp Inc', 'NETAPP INC']}]

In [71]:
kg.query("""
    match (com: Company), (form: Form)
    where com.cusip6 = form.cusip6
    set com.names = form.names
""")

[]

In [72]:
kg.query("""
    match (com: Company), (form: Form)
    where com.cusip6 = form.cusip6
    return com.companyName, form.names
""")

[{'com.companyName': ['Netapp Inc', 'NETAPP INC'],
  'form.names': ['Netapp Inc', 'NETAPP INC']}]

- Let's create a FILED relationship between the company and the Form-10K node

In [73]:
kg.query("""
    match (com: Company), (form: Form)
    where com.cusip6 = form.cusip6
    merge (com) -[:FILED]-> (form)
""")

[]

### Create manager nodes

- Create a manager node for companies that have filed a Form 13 to report their investment in NetApp
- Start with the single manager who filed the first Form 13 in the list


In [74]:
kg.query("""
    merge (m: Manager {managerCik: $managerParam.managerCik})      
        on create
            set m.managerName = $managerParam.managerName,
                m.managerAddress = $managerParam.managerAddress
""", params={'managerParam': first_form13})

[]

In [75]:
kg.query('match (m: Manager) return m limit 1')

[{'m': {'managerCik': '1000275',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'managerName': 'Royal Bank of Canada'}}]

- Create a uniquness constraint to avoid duplicate managers

In [76]:
kg.query("""
    create constraint unique_manager
    if not exists 
    for (m: Manager)
    require m.managerCik is unique
""")

[]

- Create a fulltext index of manager names to enable text search

In [77]:
kg.query("""
    create fulltext index fullTextManagerNames
    if not exists
    for (m: Manager)
    on each [m.managerName]
""")

[]

In [78]:
kg.query("""
    call db.index.fulltext.queryNodes("fullTextManagerNames", "royal bank") yield node, score
    return node.managerName, score
""")

[{'node.managerName': 'Royal Bank of Canada', 'score': 3.7019896507263184},
 {'node.managerName': 'Royal Fund Management, LLC',
  'score': 2.2780675888061523},
 {'node.managerName': 'ROYAL LONDON ASSET MANAGEMENT LTD',
  'score': 2.063053607940674},
 {'node.managerName': 'AMALGAMATED BANK', 'score': 1.798886775970459},
 {'node.managerName': 'COMMERCE BANK', 'score': 1.798886775970459},
 {'node.managerName': 'UMB Bank, n.a.', 'score': 1.5895916223526},
 {'node.managerName': 'Norinchukin Bank, The', 'score': 1.5895916223526},
 {'node.managerName': 'HUNTINGTON NATIONAL BANK', 'score': 1.5895916223526},
 {'node.managerName': 'Swiss National Bank', 'score': 1.5895916223526},
 {'node.managerName': 'AMERICAN NATIONAL BANK', 'score': 1.5895916223526},
 {'node.managerName': 'TORONTO DOMINION BANK', 'score': 1.5895916223526},
 {'node.managerName': 'Bangor Savings Bank', 'score': 1.5895916223526},
 {'node.managerName': 'Czech National Bank', 'score': 1.5895916223526},
 {'node.managerName': 'DEUTS

- Create nodes for all companies that filed a Form 13

In [79]:
for form in form13s:
    kg.query("""
    merge (m: Manager {managerCik: $managerParam.managerCik})      
        on create
            set m.managerName = $managerParam.managerName,
                m.managerAddress = $managerParam.managerAddress
""", params={'managerParam': form})

In [80]:
kg.query('match (m: Manager) return count(m)')

[{'count(m)': 561}]

### Create relationships between managers and companies

- Match companies with managers based on data in the Form 13
- Create an OWNS_STOCK_IN relationship between the manager and the company
- Start with the single manager who filed the first Form 13 in the list

In [81]:
kg.query("""
    match (m: Manager {managerCik: $investmentParam.managerCik}), (c: Company {cusip6: $investmentParam.cusip6})
    return m.managerName, c.companyName, $investmentParam as investment
""", params={'investmentParam': first_form13})

[{'m.managerName': 'Royal Bank of Canada',
  'c.companyName': ['Netapp Inc', 'NETAPP INC'],
  'investment': {'managerCik': '1000275',
   'shares': '842850',
   'cusip': '64110D104',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'reportCalendarOrQuarter': '2023-06-30',
   'companyName': 'NETAPP INC',
   'cusip6': '64110D',
   'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
   'managerName': 'Royal Bank of Canada',
   'value': '64395000000.0'}}]

In [82]:
kg.query("""
    match (m: Manager {managerCik: $ownsParam.managerCik}), (c: Company {cusip6: $ownsParam.cusip6})
        merge (m) -[owns:OWNS_STOCK_IN {reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter}]-> (c)
         on create 
            set owns.value = toFloat($ownsParam.value),
                owns.shares = toInteger($ownsParam.shares)
    return m.managerName, owns.reportCalendarOrQuarter, owns { .shares, .value},c.companyName
""", params={'ownsParam': first_form13})

[{'m.managerName': 'Royal Bank of Canada',
  'owns.reportCalendarOrQuarter': '2023-06-30',
  'owns': {'shares': 842850, 'value': 64395000000.0},
  'c.companyName': ['Netapp Inc', 'NETAPP INC']}]

- Create relationships between all of the managers who filed Form 13s and the company

In [83]:
for form in form13s:
    kg.query("""
    match (m: Manager {managerCik: $ownsParam.managerCik}), (c: Company {cusip6: $ownsParam.cusip6})
        merge (m) -[owns:OWNS_STOCK_IN {reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter}]-> (c)
         on create 
            set owns.value = toFloat($ownsParam.value),
                owns.shares = toInteger($ownsParam.shares)
    return m.managerName, owns.reportCalendarOrQuarter, owns { .shares, .value},c.companyName
""", params={'ownsParam': form})

In [84]:
kg.query("""
    match (:Manager) -[owns:OWNS_STOCK_IN]-> (:Company)
    return count(owns) as investments
""")

[{'investments': 561}]

In [85]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties: Chunk {chunkId: STRING, names: LIST,
formId: STRING, cik: STRING, cusip6: STRING, source: STRING,
chunkSeqId: INTEGER, text: STRING, textEmbedding: LIST,
f10kItem: STRING} Form {names: LIST, formId: STRING, cik:
STRING, cusip6: STRING, source: STRING} Company {names:
LIST, cusip6: STRING, companyName: LIST, cusip: STRING}
Manager {managerCik: STRING, managerName: STRING,
managerAddress: STRING} Relationship properties: SECTION
{f10kItem: STRING} OWNS_STOCK_IN {reportCalendarOrQuarter:
STRING, value: FLOAT, shares: INTEGER} The relationships:
(:Chunk)-[:NEXT]->(:Chunk) (:Chunk)-[:PART_OF]->(:Form)
(:Form)-[:SECTION]->(:Chunk) (:Company)-[:FILED]->(:Form)
(:Manager)-[:OWNS_STOCK_IN]->(:Company)


### Determine the number of investors

- Start by finding a form 10-K chunk, and save to use in subsequent queries

In [86]:
chunk_row = kg.query('match (chunk: Chunk) return chunk.chunkId as chunkId limit 1')[0]
ref_chunk_id = chunk_row['chunkId']
ref_chunk_id

'0000950170-23-027948-item1-chunk0046'

- Build up path from Form 10-K chunk to companies and managers

In [87]:
kg.query("""
    match (:Chunk {chunkId: $chunkIdParam}) -[:PART_OF]-> (f: Form)
    return f
""", params={'chunkIdParam': ref_chunk_id})

[{'f': {'formId': '0000950170-23-027948',
   'names': ['Netapp Inc', 'NETAPP INC'],
   'cik': '1002047',
   'cusip6': '64110D',
   'source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}}]

In [88]:
kg.query("""
    match (:Chunk {chunkId: $chunkIdParam}) -[:PART_OF]-> (f: Form),
        (c: Company) -[:FILED]-> (f)
    return f.source, c.companyName
""", params={'chunkIdParam': ref_chunk_id})

[{'f.source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm',
  'c.companyName': ['Netapp Inc', 'NETAPP INC']}]

In [89]:
kg.query("""
    match (:Chunk {chunkId: $chunkIdParam}) -[:PART_OF]-> (f: Form),
        (c: Company) -[:FILED]-> (f),
        (m: Manager) -[:OWNS_STOCK_IN]-> (com)
    return f.source, c.companyName, count(m.managerName) as numberOfInvestors
""", params={'chunkIdParam': ref_chunk_id})

[{'f.source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm',
  'c.companyName': ['Netapp Inc', 'NETAPP INC'],
  'numberOfInvestors': 561}]

In [90]:
kg.query("""
    match (:Chunk {chunkId: $chunkIdParam}) -[:PART_OF]-> (f: Form),
        (c: Company) -[:FILED]-> (f),
        (m: Manager) -[owns:OWNS_STOCK_IN]-> (com)
    return m.managerName + " owns " + owns.shares + 
        " shares of " + com.companyName + " at a value of $" + 
        apoc.number.format(toInteger(owns.value)) AS text
    limit 10
""", params={'chunkIdParam': ref_chunk_id})

[{'text': ['Royal Bank of Canada owns 842850 shares of ',
   'Netapp Inc',
   'NETAPP INC',
   ' at a value of $',
   '64,395,000,000']},
 {'text': ['SHELTON CAPITAL MANAGEMENT owns 39124 shares of ',
   'Netapp Inc',
   'NETAPP INC',
   ' at a value of $',
   '2,989,085,000']},
 {'text': ['PUBLIC EMPLOYEES RETIREMENT SYSTEM OF OHIO owns 106941 shares of ',
   'Netapp Inc',
   'NETAPP INC',
   ' at a value of $',
   '8,170,000']},
 {'text': ['WILBANKS SMITH & THOMAS ASSET MANAGEMENT LLC owns 6617 shares of ',
   'Netapp Inc',
   'NETAPP INC',
   ' at a value of $',
   '505,539,000']},
 {'text': ['DEPRINCE RACE & ZOLLO INC owns 320581 shares of ',
   'Netapp Inc',
   'NETAPP INC',
   ' at a value of $',
   '24,492,389,000']},
 {'text': ['COMMERCE BANK owns 101422 shares of ',
   'Netapp Inc',
   'NETAPP INC',
   ' at a value of $',
   '7,748,640,000']},
 {'text': ['D. E. Shaw & Co., Inc. owns 323440 shares of ',
   'Netapp Inc',
   'NETAPP INC',
   ' at a value of $',
   '24,710,816,000